In [1]:
import pandas_data_handler ,database_handler,lookups
import prehook
import lookups 
from misc_handler import return_lookup_items_as_dict

In [29]:
dict=return_lookup_items_as_dict(lookups.studies)
dict

{'framingham_study': 'csv/Framingham_Study.csv',
 'switzerland': 'csv/Switzerland.csv',
 'cleveland': 'csv/CVD_cleaned.csv',
 'disease': 'csv/Disease.csv',
 'incidence_global': 'csv/incidence_global.csv',
 'incidence_lebanon': 'csv/incidence_lebanon.csv',
 'mortality_lebanon': 'csv/mortality_lebanon.csv',
 'mortality_global': 'csv/mortality_global.csv',
 'prevalence_global': 'csv/prevalence_global.csv',
 'prevalence_lebanon': 'csv/prevalence_lebanon.csv'}

In [2]:
# database_handler.return_data_as_df(lookups.prediction.DISEASE.value,lookups.InputTypes.CSV)

In [3]:
prehook.execute_prehook(sql_command_directory_path = 'SQL_COMMANDS')

In [2]:
def enum_to_lists(enum_class):
    names = [item.name.lower() for item in enum_class]
    links = [ str(item.value) for item in enum_class]
    return names, links


In [7]:
df_title, source=enum_to_lists(lookups.studies)
df_title

['framingham_study',
 'switzerland',
 'cleveland',
 'disease',
 'incidence_global',
 'incidence_lebanon',
 'mortality_lebanon',
 'mortality_global',
 'prevalence_global',
 'prevalence_lebanon']

In [9]:
# source = [
#     'csv/Framingham_Study.csv',
#     "csv/Switzerland.csv",
#     "csv/CVD_cleaned.csv",
#     "csv/Disease.csv",
#     "csv/incidence_global.csv",
#     "csv/incidence_lebanon.csv",
#     "csv/mortality_lebanon.csv",
#     "csv/moratlity_global.csv",
#     "csv/prevalence_global.csv",
#     "csv/prevalence_lebanon.csv"
# ]
db_session=database_handler.create_connection()
source_name = lookups.DestinationDatabase.DATABASE_NAME.value

for title, file_path in zip(df_title, source):
    dst_table = f"stg_{source_name}_{title}"
    df_source_2 = database_handler.return_data_as_df(file_path, lookups.InputTypes.CSV)
    create_stmt = pandas_data_handler.return_create_statement_from_df(df_source_2, 'dw_reporting', dst_table)
    database_handler.execute_query(db_session, query= create_stmt)
    print(create_stmt)
    


CREATE TABLE IF NOT EXISTS dw_reporting.stg_heart_db_framingham_study (
RANDID BIGINT,
SEX BIGINT,
TOTCHOL FLOAT,
AGE BIGINT,
SYSBP FLOAT,
DIABP FLOAT,
CURSMOKE BIGINT,
CIGPDAY FLOAT,
BMI FLOAT,
DIABETES BIGINT,
BPMEDS FLOAT,
HEARTRTE FLOAT,
GLUCOSE FLOAT,
educ FLOAT,
PREVCHD BIGINT,
PREVAP BIGINT,
PREVMI BIGINT,
PREVSTRK BIGINT,
PREVHYP BIGINT,
TIME BIGINT,
PERIOD BIGINT,
HDLC FLOAT,
LDLC FLOAT,
DEATH BIGINT,
ANGINA BIGINT,
HOSPMI BIGINT,
MI_FCHD BIGINT,
ANYCHD BIGINT,
STROKE BIGINT,
CVD BIGINT,
HYPERTEN BIGINT,
TIMEAP BIGINT,
TIMEMI BIGINT,
TIMEMIFC BIGINT,
TIMECHD BIGINT,
TIMESTRK BIGINT,
TIMECVD BIGINT,
TIMEDTH BIGINT,
TIMEHYP BIGINT
);
CREATE TABLE IF NOT EXISTS dw_reporting.stg_heart_db_switzerland (
HeartDisease TEXT,
BMI FLOAT,
Smoking TEXT,
AlcoholDrinking TEXT,
Stroke TEXT,
PhysicalHealth BIGINT,
MentalHealth BIGINT,
DiffWalking TEXT,
Sex TEXT,
AgeCategory TEXT,
Race TEXT,
Diabetic TEXT,
PhysicalActivity TEXT,
GenHealth TEXT,
SleepTime BIGINT,
Asthma TEXT,
KidneyDisease TEXT,

In [6]:
create_stmt

'CREATE TABLE IF NOT EXISTS dw_reporting.stg_heart_db_prevalence_lebanon (\nlocation_name TEXT,\nsex_name TEXT,\nage_interval TEXT,\ncause_id BIGINT,\nyear BIGINT,\nval FLOAT\n);'

In [22]:
from enum import Enum

class Studies(Enum):
    FRAMINGHAM_STUDY = "csv/Framingham_Study.csv"
    SWITZERLAND = "csv/Switzerland.csv"
    CLEVELAND = "csv/CVD_cleaned.csv"
    DISEASE = "csv/Disease.csv"
    INCIDENCE_GLOBAL = "csv/incidence_global.csv"
    INCIDENCE_LEBANON = "csv/incidence_lebanon.csv"
    MORTALITY_LEBANON = "csv/mortality_lebanon.csv"
    MORTALITY_GLOBAL = "csv/mortality_global.csv"
    PREVALENCE_GLOBAL = "csv/prevalence_global.csv"
    PREVALENCE_LEBANON = "csv/prevalence_lebanon.csv"

def get_study_data():
    sources = ['"' + study.value + '"' for study in Studies]
    names = [study.name.lower() for study in Studies]
    return sources, names


sources, names = get_study_data()
sources = [s.replace("'", "") for s in sources]

print(sources)
print(names)


['"csv/Framingham_Study.csv"', '"csv/Switzerland.csv"', '"csv/CVD_cleaned.csv"', '"csv/Disease.csv"', '"csv/incidence_global.csv"', '"csv/incidence_lebanon.csv"', '"csv/mortality_lebanon.csv"', '"csv/mortality_global.csv"', '"csv/prevalence_global.csv"', '"csv/prevalence_lebanon.csv"']
['framingham_study', 'switzerland', 'cleveland', 'disease', 'incidence_global', 'incidence_lebanon', 'mortality_lebanon', 'mortality_global', 'prevalence_global', 'prevalence_lebanon']


In [26]:
source

['csv/Framingham_Study.csv',
 'csv/Switzerland.csv',
 'csv/CVD_cleaned.csv',
 'csv/Disease.csv',
 'csv/incidence_global.csv',
 'csv/incidence_lebanon.csv',
 'csv/mortality_lebanon.csv',
 'csv/mortality_global.csv',
 'csv/prevalence_global.csv',
 'csv/prevalence_lebanon.csv']